In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [4]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

売上データの分析を行うためには、いくつかの視点からデータを掘り下げることが有効です。以下に、いくつかの重要な分析ポイントとその結果を示します。

### 1. 売上トレンドの分析
- **時間軸の売上傾向**: 日毎、月毎、または四半期毎に売上データを集計し、売上の増減を可視化することで、季節的なトレンドやプロモーション効果を把握します。
  
### 2. カテゴリー別売上
- **カテゴリ別の売上分析**: 食品、衣服、日用品などのカテゴリー別に売上を集計し、どのカテゴリーが最も売上が高いかを分析します。これにより、重点的なマーケティング戦略が立てられます。

### 3. 商品別のパフォーマンス
- **商品の売上ランキング**: 単価や数量に基づいて、各商品の売上をランキングすることにより、人気商品や売れ筋商品を特定します。

### 4. 利益率の分析
- **利益とコストの比較**: 売上から原価を引いて利益を計算し、利益率（利益 / 売上）を分析することで、どの商品やカテゴリーが利益をより多く生んでいるかを可視化します。

### 5. 顧客の購買パターン
- **リピーター分析**: 売上データがあれば、リピーターの割合や、初回購入後の再購入に関する洞察を得ることができるかもしれません。

### 6. プロモーションの影響
- **プロモーション活動の効果測定**: 売上データが特定のプロモーション活動に関連付けられている場合、期間を定めてプロモーションの効果を確認します。

### 7. セグメンテーション
- **顧客セグメントの特定**: 販売データを基に、顧客のセグメンテーション（年齢層、購入傾向など）を行うことで、ターゲットを明確にします。

### 8. 地域別の傾向
- **地域の売上分布**: 地域ごとの売上データがあれば、地域別に分析することで特定の地域での戦略を立てることが可能です。

---

これらの分析を通じて、企業は市場におけるポジショニングの強化や、プロダクトミックスの最適化、それに基づくマーケティング戦略の策定を行うことが可能になります。具体的な数値やビジュアル化が必要であれば、データビジュアライゼーションツール（Excel、Tableau、Lookerなど）を用いるのが効果的です。


In [5]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0   売上データの分析を行うためには、いくつかの視点からデータを掘り下げることが有効です。以下に、...
1                                                    
2                                    ### 1. 売上トレンドの分析
3   - **時間軸の売上傾向**: 日毎、月毎、または四半期毎に売上データを集計し、売上の増減を...
4                                                    
5                                     ### 2. カテゴリー別売上
6   - **カテゴリ別の売上分析**: 食品、衣服、日用品などのカテゴリー別に売上を集計し、どの...
7                                                    
8                                  ### 3. 商品別のパフォーマンス
9   - **商品の売上ランキング**: 単価や数量に基づいて、各商品の売上をランキングすることに...
10                                                   
11                                      ### 4. 利益率の分析
12  - **利益とコストの比較**: 売上から原価を引いて利益を計算し、利益率（利益 / 売上）...
13                                                   
14                                   ### 5. 顧客の購買パターン
15  - **リピーター分析**: 売上データがあれば、リピーターの割合や、初回購入後の再購入に関...
16                                                   
17                          

In [6]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [7]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
